In [1]:
TRAIN_BOW = './aclImdb/train/labeledBow.feat'
TEST_BOW = './aclImdb/test/labeledBow.feat'

In [2]:
# GET TRAINING SET

train_space = []
with open(TRAIN_BOW) as fp:
    train_space = fp.readlines()
train_positive_space = [[1] + sample.split(' ')[1:] for sample in train_space if int(sample.split(' ')[0]) >=7]
train_negative_space = [[-1] + sample.split(' ')[1:] for sample in train_space if int(sample.split(' ')[0]) <=4]

In [3]:
# SAMPLE FROM TRAINING SET
from random import shuffle

SAMPLE_SIZE = 1000

# Positive
pos_indices = list(range(0,len(train_positive_space)))
shuffle(pos_indices)
train_positive_sample = [train_positive_space[i] for i in pos_indices[:SAMPLE_SIZE]]

# Write in a file
with open('./sample/positive_train_indices.txt', 'w') as fp:
    for line in pos_indices[:SAMPLE_SIZE]:
        fp.write(str(line)+'\n')

# Negative
neg_indices = list(range(0,len(train_negative_space)))
shuffle(neg_indices)
train_negative_sample = [train_negative_space[i] for i in neg_indices[:SAMPLE_SIZE]]

# Write in a file
with open('./sample/negative_train_indices.txt', 'w') as fp:
    for line in neg_indices[:SAMPLE_SIZE]:
        fp.write(str(line)+'\n')

training_set = train_positive_sample + train_negative_sample
shuffle(training_set)

In [4]:
# TF-IDF (Optional)
# from collections import defaultdict
# import math
# """
# In the dataset, we have each feature in the form of A:B, where:
#     A is the index of the word in the vocabulary
#     B is the frequency of the word in the instance

# In other words, we are given the tf of the words. Now we'll find the idf values for the features
# Hence getting the tf-idf values as features.
# """

# N = len(training_set)

# vocab_idf = defaultdict(float)

# for instance in training_set:
#     for features in instance:
#         feature = features.split(':')
#         vocab_idf[feature[0]] += 1

# for key, value in vocab_idf.items():
#     vocab_idf[key] = math.log(N/value)

In [5]:
# Create the training set
from word2bin import word2bin
import numpy as np
# Get the mappings to the original features
w2b, bins = word2bin()

train_matrix = np.zeros((len(training_set), len(bins)))
train_labels = np.array([i[0] for i in training_set])

for i,instance in enumerate(training_set):
    for features in instance[1:]:
        feature = list(map(int,features.split(':')))
        if feature[0] not in w2b.keys(): continue
        b_index = w2b[feature[0]]
        train_matrix[i,b_index] += feature[1]



In [6]:
# Save the dataset
import pandas as pd

# Train
df = pd.DataFrame(train_matrix)
df.to_csv('./data/train.csv', header=None, index = False)
# Train Labels
df = pd.DataFrame(train_labels)
df.to_csv('./data/train_labels.csv', header=None, index = False)